In [184]:
#import libraries
import numpy as np
import pandas as pd
import tweepy
import requests
import json
import re

# Gather

In [185]:
df_twitter_enhanced = pd.read_csv('twitter-archive-enhanced.csv') # read twitter-arcieve-enhanced.csv into df_twitter_enhanced
df_twitter_enhanced # display the dataset

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,None,None,None,None
9,89024025534

In [186]:
df_twitter_enhanced.info() # take a look at rows and columns of dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [187]:
#download the image_prediction.tsv programatically from the given udacity's url 
dogs_images = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
with open('image_prediction.tsv','wb') as file:
    file.write(dogs_images.content)
 

In [188]:
df_dogs_images = pd.read_csv('image_prediction.tsv',sep = '\t')   # read the image_prediction.tsv into df_dogs_images
df_dogs_images.head() # see the top 5 rows

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [189]:
df_dogs_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [247]:
# get your consumer key, consumer secret ,access token and access secret from twitter to get WeRAteDogs dataset
consumer_key = 'MY CONSUMER KEY'
consumer_secret = 'MY CONSUMER SECRET'
access_token = 'MY ACCESS TOKEN'
access_secret = 'MY ACCESS SECRET'

In [191]:
# Variables created for tweepy query
auth = tweepy.OAuthHandler(consumer_key,consumer_secret) 
auth.set_access_token(access_token,access_secret)
api = tweepy.API(auth)

In [11]:
# write the json material in tweet_json.txt 
with open('tweet_json.txt','a') as file:
    for every_tweet in df_twitter_enhanced.tweet_id:
        try:
            tweet = api.get_status(every_tweet , tweet_mode = 'extended')
            json.dump(tweet._json, file)
            file.write('\n')
        except:
            continue


In [192]:
tweets_list = [] #make an empty list
#load the json material in tweet_json.txt 
with open('tweet_json.txt', "r") as json_file:
    for each in json_file:
        try:
            tweet = json.loads(each)
            tweets_list.append(tweet)
        except:
            continue

In [193]:
# Create tweet_info DataFrame
tweet_json = pd.DataFrame()

In [194]:
# make columns in new dataframe and fill the data
tweet_json['id'] = list(map(lambda tweet: tweet['id'], tweets_list))
tweet_json['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_list))
tweet_json['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_list))

In [195]:
tweet_json.head()

,id,retweet_count,favorite_count
0,892420643555336193,8636,38952
1,892177421306343426,6343,33351
2,891815181378084864,4209,25125
3,891689557279858688,8755,42298
4,891327558926688256,9517,40458


In [196]:
tweet_json.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
id                2345 non-null int64
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


# ASSESSING



In [197]:
df_twitter_enhanced.text #display the text column of dataset

0       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU                                                         
1       This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV    
2       This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB                     
3       This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ                                                               
4       This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f    
5       Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tu

In [198]:
sum(df_twitter_enhanced.expanded_urls.isnull()) # check null values in expanded urls column in df_twitter_enhanced

59

In [199]:
df_twitter_enhanced.name.value_counts() # see individual value count in name column 

None        745
a           55 
Charlie     12 
Lucy        11 
Cooper      11 
Oliver      11 
Lola        10 
Tucker      10 
Penny       10 
Bo          9  
Winston     9  
Sadie       8  
the         8  
Toby        7  
Daisy       7  
Bailey      7  
Buddy       7  
an          7  
Oscar       6  
Leo         6  
Scout       6  
Milo        6  
Koda        6  
Jack        6  
Bella       6  
Stanley     6  
Jax         6  
Rusty       6  
Dave        6  
Louis       5  
           ..  
Apollo      1  
Biden       1  
Cupid       1  
Chaz        1  
Tove        1  
Jerome      1  
Ashleigh    1  
Arlo        1  
Akumi       1  
Anthony     1  
Tebow       1  
Arya        1  
Cedrick     1  
Aja         1  
Smiley      1  
Nida        1  
Aldrick     1  
Heinrich    1  
Cora        1  
Boston      1  
Rueben      1  
Dido        1  
Richie      1  
Hector      1  
Trigger     1  
Beya        1  
Chuq        1  
Brutus      1  
Kellogg     1  
light       1  
Name: name, Length: 957,

In [200]:
df_twitter_enhanced.rating_denominator.value_counts() # see individual value count in rating_denominator column

10     2333
11     3   
50     3   
80     2   
20     2   
2      1   
16     1   
40     1   
70     1   
15     1   
90     1   
110    1   
120    1   
130    1   
150    1   
170    1   
7      1   
0      1   
Name: rating_denominator, dtype: int64

In [201]:
df_twitter_enhanced.query('rating_denominator == 0') # query dataset with rating_denominator = 0

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",NaN,NaN,NaN,NaN,960,0,None,None,None,None,None


In [202]:
df_twitter_enhanced.source.value_counts() # check value counts of source

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                        91  
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     33  
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>    11  
Name: source, dtype: int64

# Quality
### 1.retweet columns present. 
### 2. dog names are not correct.
### 3. Rating numerator with decimals do  not show full value.
### 4. Source not clearly visible.
### 5. doggo,floofer,pupper,puppo have none value(missing)
### 6. text is not fully visible
### 7. Tweet ID 835246439529840640  has a  0 denominator rating.
### 8.  missing tweet images(expanded url).
### 9. wrong data types of columns.

# Tideness
### 1. tweet_json  has column name id
### 2. join all 3 dataframes
### 3. dog category variable is in 4 columns


# CLEANING

In [203]:
# make copy of dataframes and then clean it.
df_twitter_enhanced_clean = df_twitter_enhanced.copy()
df_dogs_images_clean = df_dogs_images.copy()
tweet_json_clean = tweet_json.copy()

# Define

## rename id column of tweet_json to tweet_id

# Code

In [204]:
tweet_json_clean = tweet_json_clean.rename(columns = {'id':'tweet_id'}) #rename id col to tweet id of tweet_json_clean

# Test

In [205]:
tweet_json_clean.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8636,38952
1,892177421306343426,6343,33351
2,891815181378084864,4209,25125
3,891689557279858688,8755,42298
4,891327558926688256,9517,40458


# Define

## join df_twitter_enhanced_clean , df_dogs_images and tweet_json

# Code

In [206]:
# merge df_twitter_enhanced_clean with df_dogs_images
df_twitter_enhanced_clean = pd.merge(left=df_twitter_enhanced_clean, right=df_dogs_images, left_on='tweet_id', right_on='tweet_id', how='inner')

In [207]:
#merge df_twitter_enhanced_clean with tweet_json_clean
df_twitter_enhanced_clean = pd.merge(left = df_twitter_enhanced_clean, right = tweet_json_clean, left_on = 'tweet_id', right_on = 'tweet_id',how = 'inner')

# Test

In [208]:
df_twitter_enhanced_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2069 entries, 0 to 2068
Data columns (total 30 columns):
tweet_id                      2069 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2069 non-null object
source                        2069 non-null object
text                          2069 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2069 non-null object
rating_numerator              2069 non-null int64
rating_denominator            2069 non-null int64
name                          2069 non-null object
doggo                         2069 non-null object
floofer                       2069 non-null object
pupper                        2069 non-null object
puppo                         2069 non-null object
jpg_url                       2069 

# Define

## dog stage variable is in 4 columns,convert in one column dog category

# Code

In [209]:
#extraxt keywords from text column and convert 4 dog categories to one ['dog_category']
df_twitter_enhanced_clean['dog_category'] = df_twitter_enhanced_clean['text'].str.extract('(puppo|pupper|floofer|doggo)', expand=True)

In [210]:
# drop unneeded columns of dog category
df_twitter_enhanced_clean = df_twitter_enhanced_clean.drop(['puppo','pupper','floofer','doggo'],axis = 1)

# Test

In [211]:
df_twitter_enhanced_clean.head(20) #display top 20 rows of df_twitter-enhanced_clean

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,...,False,bagel,0.085851,False,banana,0.076110,False,8636,38952,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,...,True,Pekinese,0.090647,True,papillon,0.068957,True,6343,33351,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,...,True,malamute,0.078253,True,kelpie,0.031379,True,4209,25125,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,...,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,8755,42298,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",...,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,9517,40458,NaN
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,...,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,3153,20292,NaN
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",...,True,Border_collie,0.199287,True,ice_lolly,0.193548,False,2100,11899,NaN
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",...,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,19146,65802,NaN
8,890609185150312448,NaN,NaN,20

## define

### remove retweet columns

### code

In [212]:
#remove retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp from df_twitter_enhanced_clean
df_twitter_enhanced_clean = df_twitter_enhanced_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],axis = 1)

## test

In [213]:
df_twitter_enhanced_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,...,False,bagel,0.085851,False,banana,0.076110,False,8636,38952,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,...,True,Pekinese,0.090647,True,papillon,0.068957,True,6343,33351,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,...,True,malamute,0.078253,True,kelpie,0.031379,True,4209,25125,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,...,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,8755,42298,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,...,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,9517,40458,NaN


# Define

## Display the full text in dataframe.

# Code

In [214]:
pd.set_option('display.max_colwidth', -1) # dataframe is widened to full length

# Test

In [215]:
df_twitter_enhanced_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,...,False,bagel,0.085851,False,banana,0.076110,False,8636,38952,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,...,True,Pekinese,0.090647,True,papillon,0.068957,True,6343,33351,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,...,True,malamute,0.078253,True,kelpie,0.031379,True,4209,25125,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,...,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,8755,42298,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,...,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,9517,40458,NaN


# Define

## Correct dog names from the text column

# Code

In [216]:
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains('name is')] #see rows that have 'name is' as substring in text column 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
138,859607811541651456,NaN,NaN,2017-05-03 03:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Sorry for the lack of posts today. I came home from school and had to spend quality time with my puppo. Her name is Zoey and she's 13/10 https://t.co/BArWupFAn0,https://twitter.com/dog_rates/status/859607811541651456/photo/1,13,10,None,...,True,Irish_setter,0.024099,True,Labrador_retriever,0.019285,True,1660,19157,puppo
686,765395769549590528,NaN,NaN,2016-08-16 03:52:26 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is my dog. Her name is Zoey. She knows I've been rating other dogs. She's not happy. 13/10 no bias at all https://t.co/ep1NkYoiwB,https://twitter.com/dog_rates/status/765395769549590528/photo/1,13,10,my,...,True,Cardigan,0.330401,True,Shetland_sheepdog,0.038875,True,3795,28639,NaN
1415,682047327939461121,NaN,NaN,2015-12-30 03:55:29 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",We normally don't rate bears but this one seems nice. Her name is Thea. Appears rather fluffy. 10/10 good bear https://t.co/fZc7MixeeT,https://twitter.com/dog_rates/status/682047327939461121/photo/1,10,10,None,...,False,doormat,0.119243,False,toyshop,0.035127,False,1064,3452,NaN
1468,679736210798047232,NaN,NaN,2015-12-23 18:51:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This pup's name is Sabertooth (parents must be cool). Ears for days. Jumps unannounced. 9/10 would pet diligently https://t.co/iazoiNUviP,https://twitter.com/dog_rates/status/679736210798047232/photo/1,9,10,None,...,True,Chihuahua,0.154088,True,Cardigan,0.117688,True,885,2250,NaN
1981,667524857454854144,NaN,NaN,2015-11-20 02:08:22 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Another topnotch dog. His name is Big Jumpy Rat. Massive ass feet. Superior tail. Jumps high af. 12/10 great pup https://t.co/seESNzgsdm,https://twitter.com/dog_rates/status/667524857454854144/photo/1,12,10,None,...,False,dhole,0.092435,False,Chesapeake_Bay_retriever,0.088122,True,1172,1755,NaN
2001,667177989038297088,NaN,NaN,2015-11-19 03:10:02 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Dasani Kingfisher from Maine. His name is Daryl. Daryl doesn't like being swallowed by a panda. 8/10 https://t.co/jpaeu6LNmW,https://twitter.com/dog_rates/status/667177989038297088/photo/1,8,10,a,...,True,Chesapeake_Bay_retriever,0.176293,True,Weimaraner,0.112369,True,57,194,NaN
2026,666739327293083650,NaN,NaN,2015-11-17 22:06:57 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Lugan. He is a Bohemian Rhapsody. Very confused dog. Thinks his name is Rocky. Not amused by the snows 10/10 https://t.co/tI3uFLDHBI,https://twitter.com/dog_rates/status/666739327293083650/photo/1,10,10,Lugan,...,True,cocker_spaniel,0.165255,True,toy_poodle,0.095959,True,67,240,NaN


In [217]:
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains('named')] #see rows that have 'named is' as substring in text column 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
479,798628517273620480,NaN,NaN,2016-11-15 20:47:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This a Norwegian Pewterschmidt named Tickles. Ears for days. 12/10 I care deeply for Tickles https://t.co/0aDF62KVP7,"https://twitter.com/dog_rates/status/667509364010450944/photo/1,https://twitter.com/dog_rates/status/667509364010450944/photo/1",12,10,None,...,True,Labrador_retriever,0.119256,True,golden_retriever,8.254920e-02,True,2224,0,NaN
1576,675706639471788032,NaN,NaN,2015-12-12 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI,https://twitter.com/dog_rates/status/675706639471788032/photo/1,10,10,a,...,True,Welsh_springer_spaniel,0.002080,True,cocker_spaniel,2.013780e-03,True,104,681,NaN
1673,673636718965334016,NaN,NaN,2015-12-06 22:54:44 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Lofted Aphrodisiac Terrier named Kip. Big fan of bed n breakfasts. Fits perfectly. 10/10 would pet firmly https://t.co/gKlLpNzIl3,https://twitter.com/dog_rates/status/673636718965334016/photo/1,10,10,a,...,False,corn,0.019421,False,pug,1.904430e-02,True,398,1170,NaN
1752,671743150407421952,NaN,NaN,2015-12-01 17:30:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Tuscaloosa Alcatraz named Jacob (Yacōb). Loves to sit in swing. Stellar tongue. 11/10 look at his feet https://t.co/2IslQ8ZSc7,https://twitter.com/dog_rates/status/671743150407421952/photo/1,11,10,a,...,True,miniature_poodle,0.237067,True,swing,1.041930e-01,False,241,768,NaN
1783,671147085991960577,NaN,NaN,2015-11-30 02:01:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Helvetica Listerine named Rufus. This time Rufus will be ready for the UPS guy. He'll never expect it 9/10 https://t.co/34OhVhMkVr,https://twitter.com/dog_rates/status/671147085991960577/photo/1,9,10,a,...,True,cairn,0.440122,True,silky_terrier,5.869010e-02,True,242,701,NaN
1833,670427002554466305,NaN,NaN,2015-11-28 02:20:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute https://t.co/CtuJoLHiDo,https://twitter.com/dog_rates/status/670427002554466305/photo/1,9,10,a,...,False,toy_terrier,0.038872,True,beagle,3.226440e-03,True,174,539,NaN
1842,670361874861563904,NaN,NaN,2015-11-27 22:01:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Rich Mahogany Seltzer named Cherokee. Just got destroyed by a snowball. Isn't very happy about it. 9/10 https://t.co/98ZBi6o4dj,https://twitter.com/dog_rates/status/670361874861563904/photo/1,9,10,a,...,False,spotted_salamander,0.011068,False,bison,3.896910e-03,False,71,340,NaN
1845,670303360680108032,NaN,NaN,2015-11-27 18:09:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Speckled Cauliflower Yosemite named Hemry. He's terrified of intruder dog. Not one bit comfortable. 9/10 https://t.co/yV3Qgjh8iN,https://twitter.com/dog_rates/status/670303360680108032/photo/1,9,10,a,...,True,Cardigan,0.342806,True,guinea_pig,1.562490e-01,False,144,443,NaN
1863,669923323644657664,NaN,NaN,2015-11-26 16:59:01 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a spotted Lipitor Rumpelstiltskin named Alphred. He can't wait for the Turkey. 10/

In [218]:
text_list = list(df_twitter_enhanced_clean.text) #make list of text column 
text_list[0] # display 1st index of the list

"This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU"

In [219]:

index = 0 # to get index of rows that have name is in them
row_num = 0 # to get row number
for name in text_list:
    
    list_of_words = [] #empty list
    if index != 8: #this index is so that the last name lugan  does not gets changes.
        #this if statement extracts the next index string of 'name is' which is the name and then updates it in the df_twitter_
        #enhanced_clean according to the row index whcih is in dog_name.  
        if 'name is' in name:
            index+=1
            list_of_words = name.split()
            name_idx = list_of_words.index('name')
            dog_name = list_of_words[name_idx+2]
            
            if dog_name[-1] == '.':
                dog_name = dog_name[:-1]
            df_twitter_enhanced_clean.loc[row_num,'name'] = dog_name
        
    row_num += 1     
# this for loop does the same thing as prevoius it extracts the name of dog after 'named' and updates the df_twitter_enhanced_clean
# at row_num with dog_named.
index_named = 0
row_num_named = 0
for named in text_list:
    list_of_words_named = []
    if 'named' in named:
            index_named+=1
            list_of_words_named = named.split()
            named_idx = list_of_words_named.index('named')
            dog_named = list_of_words_named[named_idx+1]
            
            if dog_named[-1] == '.':
                dog_named = dog_named[:-1]
            df_twitter_enhanced_clean.loc[row_num_named,'name'] = dog_named
        
    row_num_named += 1     
    
        
        

In [220]:
# now replace name which are not present in text column ,these are to be done manually 
df_twitter_enhanced_clean['name'].replace("infuriating",'None', inplace=True) 
df_twitter_enhanced_clean['name'].replace("light",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("life",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("an",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("mad",'None', inplace=True) 
df_twitter_enhanced_clean['name'].replace("not",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("one",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("my",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("O",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("a",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace('the', 'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("such","None", inplace=True)
df_twitter_enhanced_clean['name'].replace("actually",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("just",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("getting",'None', inplace=True) 
df_twitter_enhanced_clean['name'].replace("by",'None', inplace=True)
df_twitter_enhanced_clean['name'].replace("this",'None', inplace=True) 
df_twitter_enhanced_clean['name'].replace("very",'None', inplace=True) 
df_twitter_enhanced_clean['name'].replace("quite","None", inplace=True)
df_twitter_enhanced_clean['name'].replace("old",'None', inplace=True) 
df_twitter_enhanced_clean['name'].replace("all",'None', inplace=True) 



In [221]:
df_twitter_enhanced_clean['name'].replace('None',np.NaN, inplace=True)  # replace none values NaN

# Test

In [222]:
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains('named')] # names are corrected with text that contains 'named'

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
479,798628517273620480,NaN,NaN,2016-11-15 20:47:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This a Norwegian Pewterschmidt named Tickles. Ears for days. 12/10 I care deeply for Tickles https://t.co/0aDF62KVP7,"https://twitter.com/dog_rates/status/667509364010450944/photo/1,https://twitter.com/dog_rates/status/667509364010450944/photo/1",12,10,Tickles,...,True,Labrador_retriever,0.119256,True,golden_retriever,8.254920e-02,True,2224,0,NaN
1576,675706639471788032,NaN,NaN,2015-12-12 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI,https://twitter.com/dog_rates/status/675706639471788032/photo/1,10,10,Wylie,...,True,Welsh_springer_spaniel,0.002080,True,cocker_spaniel,2.013780e-03,True,104,681,NaN
1673,673636718965334016,NaN,NaN,2015-12-06 22:54:44 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Lofted Aphrodisiac Terrier named Kip. Big fan of bed n breakfasts. Fits perfectly. 10/10 would pet firmly https://t.co/gKlLpNzIl3,https://twitter.com/dog_rates/status/673636718965334016/photo/1,10,10,Kip,...,False,corn,0.019421,False,pug,1.904430e-02,True,398,1170,NaN
1752,671743150407421952,NaN,NaN,2015-12-01 17:30:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Tuscaloosa Alcatraz named Jacob (Yacōb). Loves to sit in swing. Stellar tongue. 11/10 look at his feet https://t.co/2IslQ8ZSc7,https://twitter.com/dog_rates/status/671743150407421952/photo/1,11,10,Jacob,...,True,miniature_poodle,0.237067,True,swing,1.041930e-01,False,241,768,NaN
1783,671147085991960577,NaN,NaN,2015-11-30 02:01:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Helvetica Listerine named Rufus. This time Rufus will be ready for the UPS guy. He'll never expect it 9/10 https://t.co/34OhVhMkVr,https://twitter.com/dog_rates/status/671147085991960577/photo/1,9,10,Rufus,...,True,cairn,0.440122,True,silky_terrier,5.869010e-02,True,242,701,NaN
1833,670427002554466305,NaN,NaN,2015-11-28 02:20:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute https://t.co/CtuJoLHiDo,https://twitter.com/dog_rates/status/670427002554466305/photo/1,9,10,Spork,...,False,toy_terrier,0.038872,True,beagle,3.226440e-03,True,174,539,NaN
1842,670361874861563904,NaN,NaN,2015-11-27 22:01:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Rich Mahogany Seltzer named Cherokee. Just got destroyed by a snowball. Isn't very happy about it. 9/10 https://t.co/98ZBi6o4dj,https://twitter.com/dog_rates/status/670361874861563904/photo/1,9,10,Cherokee,...,False,spotted_salamander,0.011068,False,bison,3.896910e-03,False,71,340,NaN
1845,670303360680108032,NaN,NaN,2015-11-27 18:09:09 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Speckled Cauliflower Yosemite named Hemry. He's terrified of intruder dog. Not one bit comfortable. 9/10 https://t.co/yV3Qgjh8iN,https://twitter.com/dog_rates/status/670303360680108032/photo/1,9,10,Hemry,...,True,Cardigan,0.342806,True,guinea_pig,1.562490e-01,False,144,443,NaN
1863,669923323644657664,NaN,NaN,2015-11-26 16:59:01 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a spotted Lipitor Rumpelstiltskin named Alphred. H

In [223]:
# names are corrected with text that contains 'name is'
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains('name is')] 


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
138,859607811541651456,NaN,NaN,2017-05-03 03:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Sorry for the lack of posts today. I came home from school and had to spend quality time with my puppo. Her name is Zoey and she's 13/10 https://t.co/BArWupFAn0,https://twitter.com/dog_rates/status/859607811541651456/photo/1,13,10,Zoey,...,True,Irish_setter,0.024099,True,Labrador_retriever,0.019285,True,1660,19157,puppo
686,765395769549590528,NaN,NaN,2016-08-16 03:52:26 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is my dog. Her name is Zoey. She knows I've been rating other dogs. She's not happy. 13/10 no bias at all https://t.co/ep1NkYoiwB,https://twitter.com/dog_rates/status/765395769549590528/photo/1,13,10,Zoey,...,True,Cardigan,0.330401,True,Shetland_sheepdog,0.038875,True,3795,28639,NaN
1415,682047327939461121,NaN,NaN,2015-12-30 03:55:29 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",We normally don't rate bears but this one seems nice. Her name is Thea. Appears rather fluffy. 10/10 good bear https://t.co/fZc7MixeeT,https://twitter.com/dog_rates/status/682047327939461121/photo/1,10,10,Thea,...,False,doormat,0.119243,False,toyshop,0.035127,False,1064,3452,NaN
1468,679736210798047232,NaN,NaN,2015-12-23 18:51:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This pup's name is Sabertooth (parents must be cool). Ears for days. Jumps unannounced. 9/10 would pet diligently https://t.co/iazoiNUviP,https://twitter.com/dog_rates/status/679736210798047232/photo/1,9,10,Sabertooth,...,True,Chihuahua,0.154088,True,Cardigan,0.117688,True,885,2250,NaN
1981,667524857454854144,NaN,NaN,2015-11-20 02:08:22 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Another topnotch dog. His name is Big Jumpy Rat. Massive ass feet. Superior tail. Jumps high af. 12/10 great pup https://t.co/seESNzgsdm,https://twitter.com/dog_rates/status/667524857454854144/photo/1,12,10,Big,...,False,dhole,0.092435,False,Chesapeake_Bay_retriever,0.088122,True,1172,1755,NaN
2001,667177989038297088,NaN,NaN,2015-11-19 03:10:02 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Dasani Kingfisher from Maine. His name is Daryl. Daryl doesn't like being swallowed by a panda. 8/10 https://t.co/jpaeu6LNmW,https://twitter.com/dog_rates/status/667177989038297088/photo/1,8,10,Daryl,...,True,Chesapeake_Bay_retriever,0.176293,True,Weimaraner,0.112369,True,57,194,NaN
2026,666739327293083650,NaN,NaN,2015-11-17 22:06:57 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Lugan. He is a Bohemian Rhapsody. Very confused dog. Thinks his name is Rocky. Not amused by the snows 10/10 https://t.co/tI3uFLDHBI,https://twitter.com/dog_rates/status/666739327293083650/photo/1,10,10,Rocky,...,True,cocker_spaniel,0.165255,True,toy_poodle,0.095959,True,67,240,NaN


In [224]:
df_twitter_enhanced_clean.name.value_counts() #value counts of names

Charlie     11
Tucker      10
Penny       10
Lucy        10
Oliver      10
Cooper      10
Bo          8 
Lola        8 
Winston     8 
Sadie       8 
Toby        7 
Daisy       7 
Bella       6 
Jax         6 
Scout       6 
Milo        6 
Koda        6 
Stanley     6 
Dave        6 
Bailey      6 
Rusty       6 
Leo         5 
Zoey        5 
Chester     5 
Alfie       5 
Oscar       5 
Buddy       5 
Louis       5 
Larry       5 
Phil        4 
           .. 
Ralphé      1 
Corey       1 
Ebby        1 
Edd         1 
Duddles     1 
Chuq        1 
Beya        1 
Trigger     1 
Smiley      1 
Fiji        1 
Keet        1 
Lulu        1 
Pippin      1 
Anthony     1 
Tebow       1 
Arya        1 
Cedrick     1 
Aja         1 
Jamesy      1 
Hector      1 
Carly       1 
Nida        1 
Aldrick     1 
Heinrich    1 
Dunkin      1 
Akumi       1 
Cora        1 
Rueben      1 
Dido        1 
Carter      1 
Name: name, Length: 939, dtype: int64

# Define

## rectifying rating numerator with decimal that do not  show full value.

# Code

In [225]:
# to remove unneccessary warning ,updates etc.
import warnings
warnings.simplefilter('ignore')

In [226]:
# to see rows that have more than one numerator/denominator value in text col.
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains( r"(\d+\.?\d*\/\d+\.?\d*\D+\d+\.?\d*\/\d+\.?\d*)")]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
450,802624713319034886,NaN,NaN,2016-11-26 21:26:58 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: ""Yep... just as I suspected. You're not flossing."" 12/10 and 11/10 for the pup not flossing https://t.co/SuXcI9B7pQ","https://twitter.com/dog_rates/status/777684233540206592/photo/1,https://twitter.com/dog_rates/status/777684233540206592/photo/1",12,10,NaN,...,True,golden_retriever,0.162850,True,otterhound,0.110921,True,3315,0,NaN
611,777684233540206592,NaN,NaN,2016-09-19 01:42:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","""Yep... just as I suspected. You're not flossing."" 12/10 and 11/10 for the pup not flossing https://t.co/SuXcI9B7pQ",https://twitter.com/dog_rates/status/777684233540206592/photo/1,12,10,NaN,...,True,golden_retriever,0.162850,True,otterhound,0.110921,True,3315,12288,NaN
821,747600769478692864,NaN,NaN,2016-06-28 01:21:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bookstore and Seaweed. Bookstore is tired and Seaweed is an asshole. 10/10 and 7/10 respectively https://t.co/eUGjGjjFVJ,"https://twitter.com/dog_rates/status/747600769478692864/photo/1,https://twitter.com/dog_rates/status/747600769478692864/photo/1",10,10,Bookstore,...,True,Weimaraner,0.054431,True,Labrador_retriever,0.043268,True,604,2507,NaN
872,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,NaN,...,True,kuvasz,0.101286,True,Labrador_retriever,0.023785,True,14681,37313,NaN
963,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20,NaN,...,True,Greater_Swiss_Mountain_dog,0.126131,True,Weimaraner,0.085297,True,1721,4396,NaN
997,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,https://twitter.com/dog_rates/status/716439118184652801/photo/1,50,50,Bluebert,...,True,malamute,0.317053,True,Eskimo_dog,0.273419,True,241,2527,NaN
1013,714258258790387713,NaN,NaN,2016-03-28 01:10:13 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Travis and Flurp. Travis is pretty chill but Flurp can't lie down properly. 10/10 &amp; 8/10\nget it together Flurp https://t.co/Akzl5ynMmE,https://twitter.com/dog_rates/status/714258258790387713/photo/1,10,10,Travis,...,True,Chesapeake_Bay_retriever,0.101834,True,beagle,0.101294,True,788,3218,NaN
1135,703356393781329922,NaN,NaN,2016-02-26 23:10:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Socks. That water pup w the super legs just splashed him. Socks did not appreciate that. 9/10 and 2/10 https://t.co/8rc5I22bBf,https://twitter.com/dog_rates/status/703356393781329922/photo/1,9,10,Socks,...,True,collie,0.097364,True,English_springer,0.003037,True,424,2046,NaN
1225,695064344191721472,NaN,NaN,2016-02-04 02:00:27 +0000,"<a

In [227]:
# list to save the text where rating was extracted wrong.
wrong_ratings = ['After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ', 
 'Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a', 
 'This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq', 
 'This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5',
 'This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv']

In [228]:
#here we fix rating_numerator that are wrongly extracted and set its rating_denominator to 10
for each in wrong_ratings:
    row = df_twitter_enhanced_clean.text == each
    df_twitter_enhanced_clean.loc[row, 'rating_numerator'] = re.findall(r"\d+\.?\d*\/\d+\.?\d*\D+(\d+\.?\d*)\/\d+\.?\d*", each)
    df_twitter_enhanced_clean.loc[row, 'rating_denominator'] = 10

In [229]:
# View tweets with decimals in rating in 'text' column 
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
40,883482846933004288,NaN,NaN,2017-07-08 00:28:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948","https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",5,10,Bella,...,True,Labrador_retriever,0.032409,True,kuvasz,0.005501,True,10098,46129,NaN
553,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",https://twitter.com/dog_rates/status/786709082849828864/photo/1,75,10,Logan,...,True,Persian_cat,0.122978,False,chow,0.102654,True,6893,19940,NaN
609,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,https://twitter.com/dog_rates/status/778027034220126208/photo/1,27,10,Sophie,...,True,cocker_spaniel,0.015950,True,Lhasa,0.006519,True,1834,7177,NaN
1447,680494726643068929,NaN,NaN,2015-12-25 21:06:00 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,https://twitter.com/dog_rates/status/680494726643068929/photo/1,26,10,NaN,...,True,Samoyed,0.111622,True,Great_Pyrenees,0.064061,True,531,1839,pupper


In [230]:
# Change datatype of rating_numerator and denominator to float
df_twitter_enhanced_clean['rating_numerator'] = df_twitter_enhanced_clean['rating_numerator'].astype('float')
df_twitter_enhanced_clean['rating_denominator'] = df_twitter_enhanced_clean['rating_denominator'].astype('float')

In [231]:
# Set correct numerators for these tweets
df_twitter_enhanced_clean.loc[(df_twitter_enhanced_clean['tweet_id'] == 883482846933004288) & (df_twitter_enhanced_clean['rating_numerator'] == 5), ['rating_numerator']] = 13.5
df_twitter_enhanced_clean.loc[(df_twitter_enhanced_clean['tweet_id'] == 786709082849828864) & (df_twitter_enhanced_clean['rating_numerator'] == 75), ['rating_numerator']] = 9.75
df_twitter_enhanced_clean.loc[(df_twitter_enhanced_clean['tweet_id'] == 778027034220126208) & (df_twitter_enhanced_clean['rating_numerator'] == 27), ['rating_numerator']] = 11.27
df_twitter_enhanced_clean.loc[(df_twitter_enhanced_clean['tweet_id'] == 680494726643068929) & (df_twitter_enhanced_clean['rating_numerator'] == 26), ['rating_numerator']] = 11.26

# Test

In [232]:
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.isin(wrong_ratings)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
872,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ","https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",14.0,10.0,NaN,...,True,kuvasz,0.101286,True,Labrador_retriever,0.023785,True,14681,37313,NaN
963,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,https://twitter.com/dog_rates/status/722974582966214656/photo/1,13.0,10.0,NaN,...,True,Greater_Swiss_Mountain_dog,0.126131,True,Weimaraner,0.085297,True,1721,4396,NaN
997,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,https://twitter.com/dog_rates/status/716439118184652801/photo/1,11.0,10.0,Bluebert,...,True,malamute,0.317053,True,Eskimo_dog,0.273419,True,241,2527,NaN
1401,682962037429899265,NaN,NaN,2016-01-01 16:30:13 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5,https://twitter.com/dog_rates/status/682962037429899265/photo/1,10.0,10.0,Darrel,...,False,Chihuahua,0.155207,True,loupe,0.153598,False,18499,39880,NaN
2048,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,https://twitter.com/dog_rates/status/666287406224695296/photo/1,9.0,10.0,NaN,...,True,toy_poodle,0.063064,True,miniature_poodle,0.025581,True,68,151,NaN


In [233]:
df_twitter_enhanced_clean[df_twitter_enhanced_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category
40,883482846933004288,NaN,NaN,2017-07-08 00:28:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948","https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",13.50,10.0,Bella,...,True,Labrador_retriever,0.032409,True,kuvasz,0.005501,True,10098,46129,NaN
553,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",https://twitter.com/dog_rates/status/786709082849828864/photo/1,9.75,10.0,Logan,...,True,Persian_cat,0.122978,False,chow,0.102654,True,6893,19940,NaN
609,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,https://twitter.com/dog_rates/status/778027034220126208/photo/1,11.27,10.0,Sophie,...,True,cocker_spaniel,0.015950,True,Lhasa,0.006519,True,1834,7177,NaN
1447,680494726643068929,NaN,NaN,2015-12-25 21:06:00 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,https://twitter.com/dog_rates/status/680494726643068929/photo/1,11.26,10.0,NaN,...,True,Samoyed,0.111622,True,Great_Pyrenees,0.064061,True,531,1839,pupper


# Define

## remove rows with missing tweet images(expanded_urls).

# Code

In [234]:
# query expanded_urls that are not null
df_twitter_enhanced_clean = df_twitter_enhanced_clean[df_twitter_enhanced_clean.expanded_urls.notnull()]

# Test

In [235]:
sum(df_twitter_enhanced_clean.expanded_urls.isnull()) #null expanded_urls is 0


0

# Define

## remove anchor tags,href to make source of usage clear .

# Code

In [236]:
# extract the source that is between the anchor tags
df_twitter_enhanced_clean.source = df_twitter_enhanced_clean.source.str.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>','Twitter for iPhone')
df_twitter_enhanced_clean.source = df_twitter_enhanced_clean.source.str.replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>','Vine - make a Scene')
df_twitter_enhanced_clean.source = df_twitter_enhanced_clean.source.str.replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>','Twitter Web Client')
df_twitter_enhanced_clean.source = df_twitter_enhanced_clean.source.str.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>','Tweet Deck')

# Test

In [237]:
df_twitter_enhanced_clean.source.value_counts() #extracted source values

Twitter for iPhone    2028
Twitter Web Client    30  
Tweet Deck            11  
Name: source, dtype: int64

# Define

## replace none value in doggo,floofer,pupper and puppo to Nan.

# Code

In [238]:
#none value replaced with Nan
df_twitter_enhanced_clean.dog_category = df_twitter_enhanced_clean.dog_category.replace('None', np.NaN)


# Test

In [239]:
df_twitter_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2069 entries, 0 to 2068
Data columns (total 24 columns):
tweet_id                 2069 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                2069 non-null object
source                   2069 non-null object
text                     2069 non-null object
expanded_urls            2069 non-null object
rating_numerator         2069 non-null float64
rating_denominator       2069 non-null float64
name                     1428 non-null object
jpg_url                  2069 non-null object
img_num                  2069 non-null int64
p1                       2069 non-null object
p1_conf                  2069 non-null float64
p1_dog                   2069 non-null bool
p2                       2069 non-null object
p2_conf                  2069 non-null float64
p2_dog                   2069 non-null bool
p3                       2069 non-null object
p3_conf                

# Define

## remove Tweet ID 835246439529840640 as it has rating denominator as 0.

# Code

In [240]:
df_twitter_enhanced_clean = df_twitter_enhanced_clean.query('rating_denominator != 0') # removing row with denominator as 0

# Test

In [241]:
df_twitter_enhanced_clean.rating_denominator.value_counts()

10.0     2056
50.0     2   
80.0     2   
150.0    1   
110.0    1   
130.0    1   
70.0     1   
170.0    1   
120.0    1   
40.0     1   
90.0     1   
7.0      1   
Name: rating_denominator, dtype: int64

# Define

## rectify wrong datatypes

## Code

In [242]:
#datatypes changed to respective correctives
df_twitter_enhanced_clean['timestamp'] = pd.to_datetime(df_twitter_enhanced_clean['timestamp'])
df_twitter_enhanced_clean['tweet_id'] = df_twitter_enhanced_clean['tweet_id'].astype('str')
df_twitter_enhanced_clean['in_reply_to_status_id'] = df_twitter_enhanced_clean['in_reply_to_status_id'].astype('str')
df_twitter_enhanced_clean['in_reply_to_user_id'] = df_twitter_enhanced_clean['in_reply_to_user_id'].astype('str')
df_twitter_enhanced_clean['category'] = df_twitter_enhanced_clean['dog_category'].astype('category')


# Test

In [243]:
df_twitter_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2069 entries, 0 to 2068
Data columns (total 25 columns):
tweet_id                 2069 non-null object
in_reply_to_status_id    2069 non-null object
in_reply_to_user_id      2069 non-null object
timestamp                2069 non-null datetime64[ns]
source                   2069 non-null object
text                     2069 non-null object
expanded_urls            2069 non-null object
rating_numerator         2069 non-null float64
rating_denominator       2069 non-null float64
name                     1428 non-null object
jpg_url                  2069 non-null object
img_num                  2069 non-null int64
p1                       2069 non-null object
p1_conf                  2069 non-null float64
p1_dog                   2069 non-null bool
p2                       2069 non-null object
p2_conf                  2069 non-null float64
p2_dog                   2069 non-null bool
p3                       2069 non-null object
p3_conf     

# Store

In [244]:
df_twitter_enhanced_clean.to_csv('twitter_archive_master.csv',index = False , encoding = 'utf-8') # storring df_twitter_enhanced_cleanto twitter-archieve-master.csv

In [245]:
twitter = pd.read_csv('twitter_archive_master.csv') # reading twitter_archieve_master.csv

In [246]:
twitter.head() # display first 5 rows of dataset

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,dog_category,category
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13.0,10.0,Phineas,...,bagel,0.085851,False,banana,0.076110,False,8636,38952,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13.0,10.0,Tilly,...,Pekinese,0.090647,True,papillon,0.068957,True,6343,33351,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12.0,10.0,Archie,...,malamute,0.078253,True,kelpie,0.031379,True,4209,25125,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13.0,10.0,Darla,...,Labrador_retriever,0.168086,True,spatula,0.040836,False,8755,42298,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12.0,10.0,Franklin,...,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,9517,40458,NaN,NaN
